In [86]:
import constti
import Brr_functions
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
from pathlib import Path

def inputFPL():
    team_number = 20
    url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
    url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
    url3 = "https://fantasy.premierleague.com/api/event/6/live/"
    url4 = "https://fantasy.premierleague.com/api/fixtures"

    #Kills unfinished matches
    def is_finished(n):
        if n=='':
            return False
        else:
            a = Fixtures[Fixtures['id']==n]['finished']
            #print(a)
            return a.bool()
        
        
    p1 = requests.get(url1)
    #page1 = BeautifulSoup(p1.text)
    #data1 = str(page1.p)[3:-4]
    data1 = p1.text
    
    d1 = json.loads(data1)
    bigTable = pd.DataFrame(d1['elements'])
    bigTable = bigTable[['team', 'element_type', 'web_name', 'goals_scored', 'assists', 'bonus', 'event_points', 'total_points', 
                   'saves', 'own_goals', 'clean_sheets', 'penalties_missed', 'penalties_saved', 'yellow_cards', 'red_cards', 
                   'minutes', 'bps', 'creativity', 'threat', 'ict_index', 'influence',
                   'value_season', 'form', 'value_form', 'points_per_game', 
                   'goals_conceded', 
                   'in_dreamteam', 'dreamteam_count',
                   'now_cost', 'cost_change_event', 'cost_change_event_fall',
                   'cost_change_start', 'cost_change_start_fall', 'selected_by_percent',
                   'transfers_in_event', 'transfers_out_event', 'transfers_in', 'transfers_out', 
                   'chance_of_playing_this_round', 'chance_of_playing_next_round', 'news_added', 'news', 'status', 
                   'ep_this', 'ep_next', 'first_name', 'second_name', 'team_code', 'id', 'photo', 'special', 'squad_number', 'code']]
    bigTable['full_name'] = bigTable['first_name'] + ' ' + bigTable['second_name']
    bigTable.to_csv(Path('in/fpltable.csv'))

    p4 = requests.get(url4)
    #page4 = BeautifulSoup(p4.text)
    #data4 = str(page4.p)[3:-4]
    d4 = json.loads(p4.text)
    Fixtures = pd.DataFrame(d4)
    Fixtures.to_csv(Path('in/fplfixtures.csv'))
    
    firstr = len(Fixtures)+1
    lastr = 0
    for i in range(len(Fixtures)):
        if Fixtures.at[i,'finished']==True:
            firstr = min(firstr, i)
            lastr = i
    if firstr < len(Fixtures)+1:
        lastGW = int(Fixtures.at[lastr,'event'])
    else: lastGW = 0
    if lastr == len(Fixtures):
        lastGW = int(Fixtures.at[lastr,'event'])







    Gameweeks = pd.DataFrame()
    for i in range(1,2*team_number - 1):
        url = "https://fantasy.premierleague.com/api/event/" + str(i) + "/live/"
        p = requests.get(url)
        #page = BeautifulSoup(p.text)
        #data = str(page.p)[3:-4]
        d = json.loads(p.text)
        nexTour = pd.DataFrame(d['elements'])

        if not nexTour.empty:
            nt1 = pd.DataFrame(nexTour['stats'].tolist())
            nt1['id'] = nexTour['id']
            nt1['gameweek'] = i
            nt1['fixture'] = ''
            for j in nexTour.index:
                #if nexTour.at[j,'explain']==[]:
                #    nt1.at[j, 'fixture'] = ''
                if len(nexTour.at[j,'explain'])==1:
                    nt1.at[j, 'fixture'] = nexTour.at[j,'explain'][0]['fixture']
                if len(nexTour.at[j,'explain'])==2:
                    nt1.at[j, 'fixture'] = nexTour.at[j,'explain'][0]['fixture']
                    newline = nt1.loc[j].copy()
                    newline['fixture'] = nexTour.at[j,'explain'][1]['fixture']
                    #print(newline)
                    nt1 = nt1.append(newline, ignore_index=True)
                if len(nexTour.at[j,'explain'])==3:
                    nt1.at[j, 'fixture'] = nexTour.at[j,'explain'][0]['fixture']
                    newline1 = nt1.loc[j].copy()
                    newline2 = nt1.loc[j].copy()
                    newline1['fixture'] = nexTour.at[j,'explain'][1]['fixture']
                    newline2['fixture'] = nexTour.at[j,'explain'][2]['fixture']
                    #print(newline1, newline2)
                    nt1 = nt1.append(newline1, ignore_index=True)
                    nt1 = nt1.append(newline2, ignore_index=True)
                if len(nexTour.at[j,'explain'])>3:
                    print('Too many matches in Gameweek')
            #nt1['fixture'] = [nexTour.at[j,'explain'][0]['fixture'] if not nexTour.at[j,'explain']==[] \
            #                  else '' for j in nexTour.index]
            #nt1.index = nt1['gameweek']*1000+nt1['id']
            Gameweeks = Gameweeks.append(nt1, ignore_index=True)
            print(i)


    teams = dict(zip(pd.DataFrame(d1['teams'])['id'],pd.DataFrame(d1['teams'])['name']))
    players = dict(zip(bigTable['id'],bigTable['full_name']))
    teamplayers = dict(zip(bigTable['id'],bigTable['team']))

    Gameweeks['team'] = [teamplayers[i] for i in Gameweeks['id']]

    Gameweeks['threat'] = pd.to_numeric(Gameweeks['threat'])
    Gameweeks['creativity'] = pd.to_numeric(Gameweeks['creativity'])
    
    #Kill postponed matches lines

    li =[]
    for i in Gameweeks.index:
        if not is_finished(Gameweeks.at[i,'fixture']):
            li.append(i)
    Gameweeks = Gameweeks.drop(li)

    Gameweeks['team_a'] = [Fixtures[(pd.to_numeric(Fixtures['id']) == pd.to_numeric(Gameweeks.at[i,'fixture'])) & \
                                             ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                              (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_a'].values \
                                for i in Gameweeks.index]

    Gameweeks['team_a'] = [int(Gameweeks.at[i,'team_a'][0]) if len(Gameweeks.at[i,'team_a'])==1 else '' for i in Gameweeks.index]

    Gameweeks['team_h'] = [Fixtures[(pd.to_numeric(Fixtures['id']) == pd.to_numeric(Gameweeks.at[i,'fixture'])) & \
                                             ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                              (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_h'].values \
                                for i in Gameweeks.index]
    Gameweeks['team_h'] = [int(Gameweeks.at[i,'team_h'][0]) if len(Gameweeks.at[i,'team_h'])==1 else '' for i in Gameweeks.index]

    Gameweeks['teamAgainst'] = [Gameweeks.at[i,'team_a'] if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                                else Gameweeks.at[i,'team_h'] \
                               for i in Gameweeks.index]
    Gameweeks['side'] = ['home' if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                                else 'away' \
                               for i in Gameweeks.index]

    del Gameweeks['team_a']
    del Gameweeks['team_h']


    Gameweeks.to_csv(Path('in/fplgameweeks.csv'))
    return d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers

if __name__ == '__main__':
    d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers = inputFPL()
    #display(Gameweeks[(Gameweeks['fixture']==237)][['teamAgainst', 'team']])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


,teamAgainst,team
12925,10,20
12926,10,20
12927,10,20
13098,10,20
13099,10,20
13100,10,20
13101,10,20
13102,10,20
13103,10,20
13104,10,20


In [1]:
import constti
import Brr_functions
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
from pathlib import Path

if True:
    team_number = 20
    url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
    url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
    url3 = "https://fantasy.premierleague.com/api/event/6/live/"
    url4 = "https://fantasy.premierleague.com/api/fixtures"

    #Kills unfinished matches
    def is_finished(n):
        if n=='':
            return False
        else:
            a = Fixtures[Fixtures['id']==n]['finished']
            #print(a)
            return a.bool()
        
        
    p1 = requests.get(url1)
    #page1 = BeautifulSoup(p1.text)
    #data1 = str(page1.p)[3:-4]
    data1 = p1.text
    
    d1 = json.loads(data1)
    bigTable = pd.DataFrame(d1['elements'])
    bigTable = bigTable[['team', 'element_type', 'web_name', 'goals_scored', 'assists', 'bonus', 'event_points', 'total_points', 
                   'saves', 'own_goals', 'clean_sheets', 'penalties_missed', 'penalties_saved', 'yellow_cards', 'red_cards', 
                   'minutes', 'bps', 'creativity', 'threat', 'ict_index', 'influence',
                   'value_season', 'form', 'value_form', 'points_per_game', 
                   'goals_conceded', 
                   'in_dreamteam', 'dreamteam_count',
                   'now_cost', 'cost_change_event', 'cost_change_event_fall',
                   'cost_change_start', 'cost_change_start_fall', 'selected_by_percent',
                   'transfers_in_event', 'transfers_out_event', 'transfers_in', 'transfers_out', 
                   'chance_of_playing_this_round', 'chance_of_playing_next_round', 'news_added', 'news', 'status', 
                   'ep_this', 'ep_next', 'first_name', 'second_name', 'team_code', 'id', 'photo', 'special', 'squad_number', 'code']]
    bigTable['full_name'] = bigTable['first_name'] + ' ' + bigTable['second_name']
    bigTable.to_csv(Path('in/fpltable.csv'))

    p4 = requests.get(url4)
    #page4 = BeautifulSoup(p4.text)
    #data4 = str(page4.p)[3:-4]
    d4 = json.loads(p4.text)
    Fixtures = pd.DataFrame(d4)
    Fixtures.to_csv(Path('in/fplfixtures.csv'))
    
    firstr = len(Fixtures)+1
    lastr = 0
    for i in range(len(Fixtures)):
        if Fixtures.at[i,'finished']==True:
            firstr = min(firstr, i)
            lastr = i
    if firstr < len(Fixtures)+1:
        lastGW = int(Fixtures.at[lastr,'event'])
    else: lastGW = 0
    if lastr == len(Fixtures):
        lastGW = int(Fixtures.at[lastr,'event'])







    Gameweeks = pd.DataFrame()
    for i in range(1,2*team_number - 1):
        url = "https://fantasy.premierleague.com/api/event/" + str(i) + "/live/"
        p = requests.get(url)
        #page = BeautifulSoup(p.text)
        #data = str(page.p)[3:-4]
        d = json.loads(p.text)
        nexTour = pd.DataFrame(d['elements'])

        if not nexTour.empty:
            nt1 = pd.DataFrame(nexTour['stats'].tolist())
            nt1['id'] = nexTour['id']
            nt1['gameweek'] = i
            nt1['fixture'] = ''
            for j in nexTour.index:
                #if nexTour.at[j,'explain']==[]:
                #    nt1.at[j, 'fixture'] = ''
                if len(nexTour.at[j,'explain'])==1:
                    nt1.at[j, 'fixture'] = nexTour.at[j,'explain'][0]['fixture']
                if len(nexTour.at[j,'explain'])==2:
                    nt1.at[j, 'fixture'] = nexTour.at[j,'explain'][0]['fixture']
                    newline = nt1.loc[j].copy()
                    newline['fixture'] = nexTour.at[j,'explain'][1]['fixture']
                    #print(newline)
                    nt1 = nt1.append(newline, ignore_index=True)
                if len(nexTour.at[j,'explain'])==3:
                    nt1.at[j, 'fixture'] = nexTour.at[j,'explain'][0]['fixture']
                    newline1 = nt1.loc[j].copy()
                    newline2 = nt1.loc[j].copy()
                    newline1['fixture'] = nexTour.at[j,'explain'][1]['fixture']
                    newline2['fixture'] = nexTour.at[j,'explain'][2]['fixture']
                    #print(newline1, newline2)
                    nt1 = nt1.append(newline1, ignore_index=True)
                    nt1 = nt1.append(newline2, ignore_index=True)
                if len(nexTour.at[j,'explain'])>3:
                    print('Too many matches in Gameweek')
            #nt1['fixture'] = [nexTour.at[j,'explain'][0]['fixture'] if not nexTour.at[j,'explain']==[] \
            #                  else '' for j in nexTour.index]
            #nt1.index = nt1['gameweek']*1000+nt1['id']
            Gameweeks = Gameweeks.append(nt1, ignore_index=True)
            print(i)


    teams = dict(zip(pd.DataFrame(d1['teams'])['id'],pd.DataFrame(d1['teams'])['name']))
    players = dict(zip(bigTable['id'],bigTable['full_name']))
    teamplayers = dict(zip(bigTable['id'],bigTable['team']))

    Gameweeks['team'] = [teamplayers[i] for i in Gameweeks['id']]

    Gameweeks['threat'] = pd.to_numeric(Gameweeks['threat'])
    Gameweeks['creativity'] = pd.to_numeric(Gameweeks['creativity'])
    
    #Kill postponed matches lines

    li =[]
    for i in Gameweeks.index:
        if not is_finished(Gameweeks.at[i,'fixture']):
            li.append(i)
    Gameweeks = Gameweeks.drop(li)

    Gameweeks['team_a'] = [Fixtures[(pd.to_numeric(Fixtures['id']) == pd.to_numeric(Gameweeks.at[i,'fixture'])) & \
                                             ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                              (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_a'].values \
                                for i in Gameweeks.index]

    Gameweeks['team_a'] = [int(Gameweeks.at[i,'team_a'][0]) if len(Gameweeks.at[i,'team_a'])==1 else '' for i in Gameweeks.index]

    Gameweeks['team_h'] = [Fixtures[(pd.to_numeric(Fixtures['id']) == pd.to_numeric(Gameweeks.at[i,'fixture'])) & \
                                             ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                              (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_h'].values \
                                for i in Gameweeks.index]
    Gameweeks['team_h'] = [int(Gameweeks.at[i,'team_h'][0]) if len(Gameweeks.at[i,'team_h'])==1 else '' for i in Gameweeks.index]

    Gameweeks['teamAgainst'] = [Gameweeks.at[i,'team_a'] if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                                else Gameweeks.at[i,'team_h'] \
                               for i in Gameweeks.index]
    Gameweeks['side'] = ['home' if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                                else 'away' \
                               for i in Gameweeks.index]

    del Gameweeks['team_a']
    del Gameweeks['team_h']


    Gameweeks.to_csv(Path('in/fplgameweeks.csv'))
    
Gameweeks[(Gameweeks['team']==10)&(Gameweeks['gameweek']==24)]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,...,saves,threat,total_points,yellow_cards,id,gameweek,fixture,team,teamAgainst,side
12904,0,0,42,1,0.0,1,0,5.3,False,53.2,...,6,0.0,9,0,189,24,180,10,19,away
12905,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,190,24,180,10,19,away
12940,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,526,24,180,10,19,away
12941,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,530,24,180,10,19,away
12942,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,531,24,180,10,19,away
13018,0,0,35,1,26.9,1,0,5.3,False,17.4,...,0,8.0,7,1,181,24,180,10,19,away
13019,1,1,55,1,71.8,1,0,13.5,True,53.8,...,0,10.0,12,0,182,24,180,10,19,away
13020,0,0,45,1,1.8,1,0,4.4,False,40.4,...,0,2.0,8,0,183,24,180,10,19,away
13021,0,0,39,1,1.9,1,0,3.9,False,37.0,...,0,0.0,8,0,184,24,180,10,19,away
13022,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,185,24,180,10,19,away


In [28]:
url = 'https://fantasy.premierleague.com/api/element-summary/191/'
p = requests.get(url)
d = json.loads(p.text)
dd = pd.DataFrame(d['history'])
d.keys()
dd

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards
0,1,2,40,0,39.7,191,1,1,1,13.4,...,1,4,39.0,12,0,0,0,125,True,0
1,0,0,0,1,11.8,191,19,0,0,4.3,...,2,1,29.0,3,114282,156910,42628,125,False,0
2,0,3,43,0,16.6,191,24,1,2,15.5,...,1,3,67.0,15,-351541,47097,398638,125,True,0
3,1,0,15,1,38.4,191,32,0,0,12.3,...,3,0,58.0,6,243530,291554,48024,125,False,0
4,0,2,34,0,41.8,191,44,1,1,15.9,...,1,3,73.0,9,31898,124574,92676,125,True,0
5,1,0,11,0,15.3,191,53,1,0,6.6,...,2,1,36.0,5,-80384,49872,130256,125,False,0
6,0,0,2,1,13.3,191,68,0,0,8.9,...,1,0,68.0,3,48879,123616,74737,125,False,0
7,0,0,5,0,37.7,191,74,1,0,15.0,...,1,2,94.0,2,-92287,34441,126728,125,True,0
8,0,0,0,0,0.0,191,87,0,0,0.0,...,1,1,0.0,0,-601424,13726,615150,124,False,0
9,0,0,25,0,15.3,191,94,1,1,15.3,...,1,2,93.0,7,-114968,23153,138121,124,True,0


In [2]:
nexTour

""


In [10]:
url = "https://fantasy.premierleague.com/api/event/24/live/"
p = requests.get(url)
d = json.loads(p.text)
nexTour = pd.DataFrame(d['elements'])
nexTour.to_excel('nexTour.xls')
nexTour.at[594,'explain']

[{'fixture': 180,
  'stats': [{'identifier': 'minutes', 'points': 0, 'value': 0}]},
 {'fixture': 234,
  'stats': [{'identifier': 'minutes', 'points': 0, 'value': 0}]}]

In [14]:
d

{'elements': [{'id': 14,
   'stats': {'minutes': 90,
    'goals_scored': 0,
    'assists': 0,
    'clean_sheets': 0,
    'goals_conceded': 2,
    'own_goals': 0,
    'penalties_saved': 0,
    'penalties_missed': 0,
    'yellow_cards': 0,
    'red_cards': 0,
    'saves': 6,
    'bonus': 0,
    'bps': 20,
    'influence': '42.0',
    'creativity': '0.0',
    'threat': '0.0',
    'ict_index': '4.2',
    'total_points': 3,
    'in_dreamteam': False},
   'explain': [{'fixture': 238,
     'stats': [{'identifier': 'minutes', 'points': 2, 'value': 90},
      {'identifier': 'goals_conceded', 'points': -1, 'value': 2},
      {'identifier': 'saves', 'points': 2, 'value': 6}]}]},
  {'id': 27,
   'stats': {'minutes': 0,
    'goals_scored': 0,
    'assists': 0,
    'clean_sheets': 0,
    'goals_conceded': 0,
    'own_goals': 0,
    'penalties_saved': 0,
    'penalties_missed': 0,
    'yellow_cards': 0,
    'red_cards': 0,
    'saves': 0,
    'bonus': 0,
    'bps': 0,
    'influence': '0.0',
    'cre

In [16]:
p1 = requests.get(url1)

data1 = p1.text
    
d1 = json.loads(data1)
bigTable = pd.DataFrame(d1['elements'])
d1.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [21]:
d1['events']


[{'id': 1,
  'name': 'Gameweek 1',
  'deadline_time': '2019-08-09T18:00:00Z',
  'average_entry_score': 65,
  'finished': True,
  'data_checked': False,
  'highest_scoring_entry': 3493085,
  'deadline_time_epoch': 1565373600,
  'deadline_time_game_offset': 0,
  'highest_score': 142,
  'is_previous': False,
  'is_current': False,
  'is_next': False,
  'chip_plays': [{'chip_name': 'bboost', 'num_played': 128770},
   {'chip_name': '3xc', 'num_played': 271367}],
  'most_selected': 183,
  'most_transferred_in': 1,
  'top_element': 214,
  'top_element_info': {'id': 214, 'points': 20},
  'transfers_made': 0,
  'most_captained': 191,
  'most_vice_captained': 189},
 {'id': 2,
  'name': 'Gameweek 2',
  'deadline_time': '2019-08-17T10:30:00Z',
  'average_entry_score': 41,
  'finished': True,
  'data_checked': False,
  'highest_scoring_entry': 6106693,
  'deadline_time_epoch': 1566037800,
  'deadline_time_game_offset': 0,
  'highest_score': 119,
  'is_previous': False,
  'is_current': False,
  'is_